# SEIR model class experiments

Utilities for train, forecasting and plotting are now provided in nb_utils.
Documentation for the helper functions are available as docstrings.

Set the appropriate training parameters and forecast parameters in the 
corresponding cells.

train_eval   
> Train and evaluate models (currently we only support SEIR family). 
  Default configuration parameters for train and test files are 
  provided as train_config.json and test_config.json respectively.
  Search paramters, loss functions etc, are specified in the config
  files.

forecast, plot
> Generate forecasts using a trained model and plotting utilities.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import pandas as pd

from datetime import datetime, timedelta

from nb_utils import train_eval
from nb_utils import forecast
from nb_utils import plot
from nb_utils import plot_m1, plot_m2, plot_m3

## Load default configurations

In [ ]:
with open('train_config.json') as f_train, \
    open('test_config.json') as f_test, \
    open('forecast_config.json') as f_forecast:
    default_train_config = json.load(f_train)
    default_test_config = json.load(f_test)
    default_forecast_config = json.load(f_forecast)

## Set the model class

In [ ]:
model_class = 'SEIHRD'
default_train_config['model_class'] = model_class
default_test_config['model_class'] = model_class
default_forecast_config['model_class'] = model_class

## Optional: Modify training loss function

In [ ]:
variable_and_weights = [('confirmed', 0.25) , ('recovered', 0.25), ('deceased', 0.25), ('hospitalized', 0.25)]
weights = {'metric_name' : 'mape', 'variable_weights' : []}
for (variable, weight) in variable_and_weights:
    weights['variable_weights'].append({
        "variable" : variable,
        "weight" : weight
    })
default_train_config['training_loss_function'].update(weights)

## Training parameters

In [ ]:
region = ['bengaluru urban']
region_type = 'district'

In [ ]:
#Current day : Last day without the flattening of counts
current_day = datetime.now().date() - timedelta(3)

# Train1 : 1 week interval starting 2 weeks prior to current day
train1_start_date = current_day - timedelta(14)
train1_end_date = current_day - timedelta(8)
train1_run_day = train1_start_date - timedelta(1)

train1_start_date = train1_start_date.strftime("%-m/%-d/%y")
train1_end_date = train1_end_date.strftime("%-m/%-d/%y")
train1_run_day = train1_run_day.strftime("%-m/%-d/%y")

# Train2 : 1 week interval starting 1 week prior to current day
train2_start_date = current_day - timedelta(7)
train2_end_date = current_day
train2_run_day = train2_start_date - timedelta(1)

train2_start_date = train2_start_date.strftime("%-m/%-d/%y")
train2_end_date = train2_end_date.strftime("%-m/%-d/%y")
train2_run_day = train2_run_day.strftime("%-m/%-d/%y")

# Test: 1 week interval prior to current day
test_start_date = current_day - timedelta(7)
test_end_date = current_day
test_run_day = test_start_date - timedelta(1)

test_start_date = test_start_date.strftime("%-m/%-d/%y")
test_end_date = test_end_date.strftime("%-m/%-d/%y")
test_run_day = test_run_day.strftime("%-m/%-d/%y")

## train_eval on the region of interest

In [ ]:
#TODO: Update to run for all regions
params, metrics, train1_params, train2_params = train_eval(region, region_type, 
                                                           train1_start_date, train1_end_date, 
                                                           train2_start_date, train2_end_date, train2_run_day,
                                                           test_start_date, test_end_date,
                                                           default_train_config, default_test_config,
                                                           max_evals = 10, data_source = 'tracker_district_daily', 
                                                           mlflow_log = False,
                                                           name_prefix = region[0]);

## Inspect model parameters

In [ ]:
train1_params

In [ ]:
train2_params

In [ ]:
metrics

## Generate Plots

In [ ]:
%matplotlib inline
plot_m1(train1_params, train1_run_day, train1_start_date, train1_end_date, 
        test_run_day, test_start_date, test_end_date, 
        rolling_average = False, uncertainty = False, 
        plot_config = 'plot_config.json', plot_name = region[0]+'_m1.png')

In [ ]:
%matplotlib inline
plot_m2(train2_params, train1_start_date, train1_end_date, 
        test_run_day, test_start_date, test_end_date, plot_name = region[0]+'_m2.png')

In [ ]:
%matplotlib inline
forecast_start_date = (current_day + timedelta(1)).strftime("%-m/%-d/%y")
forecast_length = 30 # Forecast for 30 days
plot_m3(train2_params, train1_start_date, 
        forecast_start_date, forecast_length, plot_name = region[0]+'_m3.png')